# Data Analysis with Python and Pandas 

---

```python 

import pandas as pd # importing the Pandas library

```

---

# Data Upload

The first step of data analysis is to actually get your data in the right place. Remember that using Colab is like borrowing someone else's computer. So, in order to upload our csv (called "SternTech_UserData.csv") we need to: 

1. **Download the .zip file from the NYU Classes > Resources folder to your own computer,** 

2. **In Colab, click on the little arrow on the left-hand side of the screen,**

3. **Click on "Files" and then "Upload" to upload the unzipped .csv file,**

4. **Select "SternTech_UserData.csv" (again, the unzipped version) and click "open".**

---

### Now that I have my csv in the right place, I can "read it in" using pd.read_csv.

### First we are going to work with our 'SternTech_UserData.csv', a dataset of fake data that I created to illustrate the basic tenants of data analysis using Python and Pandas. 

### Below, we are setting our dataset equal to the variable 'df' (a commonplace variable name, standing for 'data frame'). 

``` python

df = pd.read_csv('./SternTech_UserData.csv',encoding='utf-8') # read in the csv

# you can ignore the 'encoding' piece for now, we'll get to that later on when we talk about web scraping. 

```

# Primary Analysis of our Data

``` python

pd.options.display.max_rows = 2000 # the way Jupyter Notebook tends to display the results of such queries isn't 
                                   # always helpful, but we can very easily change that.
                                   # this will ensure we can view up to 2,000 rows without seeing elipses in the UI
    
pd.options.display.max_columns = 50 # try commenting out this last line ('max_columns =50') then run the cell below
                                    # to see the difference this formatting makes 
    
```

### `df.head()` will give us the first five rows of our data frame 
### `df.tail()` will give us the last five rows 
### `df.head(15)` will give us the first fifteen rows <br>

``` python 

df.head() 

```

### `df.columns` will give us a list of all the column names in our data frame

``` python

df.columns 

```

### `df.dtypes` is going to tell us how the computer is interpreting our data (for instance, as a string, integer, float, et. cetera). Please note that in Pandas, "object" is, for all intents and purposes, the same as a "string" in Python.

``` python

df.dtypes 

```

### Let's drop that "unnamed" column because it's not going to do us any good.

``` python

df = df.drop(df.columns[[0]],axis=1)

# this is saying, "drop the first column (the 0th index)"
# "axis=1" specifies that we want to drop a column. If we used "axis=0" we would be specifying a row to drop

```

### `df.describe` is going to give us the basic statistical metrics for our data frame

``` python

df.describe(include="all")

```

### `df.count()` is going to give us a count of the non-null cells in each column

``` python

df.count()

```

### If we want to see the count of non-null cells for a particular column, we can use column indexing as such:

``` python

df['sex'].value_counts() 

```

### Now, looking back at our `df.dtypes` result, we see that our timestamp values are being stored as 'non-null object's' and not as timestamps, as we'd like. 

### Remember, in Python, how your data is being perceived (the dtype) determines what you can do with it. If we want to do any sort of timeseries analysis in the future, we're going to need to convert our time data from object to timestamp. So, let's change that using `pd.to_datetime`

``` python

df['timestamp'] = pd.to_datetime(df['timestamp'])

```

``` python

df.head() # let's check to make sure that nothing funky happened

```

# Primary Analysis of our Data, Continued

### `df.sample()` is going to give us a random row from our data frame

``` python

df.sample()

```

### To select a single column from our data frame, we can use column indexing again. 

``` python

df['age']

```

### To select multiple columns, we can use `.loc` notation. 

### Note that `.loc` notation is used when you're sorting by column names, whereas `.iloc` is used when you're sorting by index number. For instance:

``` python

df.loc[:, ['age','sex']] 

```

``` python

df.iloc[3] 

```

``` python

df.iloc[3,6] # get the value of the 7th column (ad_type) for the 4th row (3rd index)

```

### We can get the mean value of a column using `.mean()`

``` python

df['age'].mean() 

```

### We can also sort the values in our column using `df.sort_values(by=...)`

``` python

df.sort_values(by="age",ascending=False) 

# ascending=False means we want to have the greatest values at the top

```

### If we want to find any rows where a certain condition holds true, we can use column indexing as well as a comparative (such as `<` or `>` or `=`).

``` python 

df[df['age'] < 21] 

# return any rows where age < 21

```

---

# Exercise 1: How many 21 year-olds were served Culinary ads?

``` python

# your code here

```

# Solution

``` python

df[(df['age']==21) & (df['ad_type']=='Culinary')].count()

```

---

# Exercise 2: What is the most common company size in the SouthEast?

``` python

# your code here

```

# Solution

``` python

SouthEast_df = df[df['location'] == 'SouthEast']
SouthEast_df

```

``` python

SouthEast_df['company_size'].describe()

```

---

# Working with More JSON

### Moving on, let's look at a larger data set from https://data.cityofnewyork.us/Health/New-York-City-Leading-Causes-of-Death/jb7j-dtam that details the leading causes of dath in NYC.

``` python

import requests

url = 'http://data.cityofnewyork.us/api/views/jb7j-dtam/rows.json'
results = requests.get(url).json() # reading in the json just as we did with our citibike info last week

```

### Again, we're going to use the requests library to read the json from the given URL. You'll note that there are two main fields returned in the json – the "meta" that just describes the actual metadata, and the data itself. 

``` python

results.keys()

```

``` python

results['data']

```

### Now we'll create a DataFrame from our JSON again...

``` python

df = pd.DataFrame(results["data"])
df.head()

```

### And add some column names

``` python

columns = results["meta"]["view"]["columns"] # this gives us the descriptions and names for the columns
columns

```

``` python

headers = [i["fieldName"] for i in columns] # we create a list of the column names 
headers

```

``` python

df = pd.DataFrame(results["data"], columns=headers) # and then pass in a list of those column names to our df
df

```

### There's a lot of extraneous information in this dataframe, so we can drop a few of them. 

### Note that here we are passing in a list of columns that we'd like to drop, and specifying that we want to drop the columns themselves. If we said "axis=index" we would be dropping the rows themselves. 

### Also note that "inplace=True" specifies that instead of creating a new dataframe, we want to replace it with the current one (the one with fewer columns). This means that the new, smaller dataframe will persist across our entire notebook – aka, it implies that we want the change to be permanent. 

``` python

df.drop(labels = [':sid', ':position', ':meta', ':created_meta', ':updated_meta'], axis=1, inplace=True)
df

```

### It looks like our last three rows appear to be metadata and not actual data, so let's drop those rows as well.

``` python

df.drop(df.index[1094:], inplace=True)
df

```

### It's important to note that we can always rename our columns using a dictionary:

``` python

renaming_dict = {
    ':id': 'key', 
    ':created_at': 'created_at', 
    ':updated_at': 'updated_at'
}

df.rename(columns=renaming_dict, inplace=True)
df

```

### We've spoken a bit about datatypes, and why it's important that our computer is viewing data as we need it to; for instance, a string as a string, an integer as an integer.  

### Remember that 'object' is a string in this case...

``` python

df.dtypes

```

### Let's change 'year' to an integer that way we can sort by year using `pd.to_numeric`:

``` python

df["year"] = pd.to_numeric(df["year"])
df.dtypes

```

### We can also pass the `errors` command to specify what should happen if we anticipate Pandas is going to object to one of our changes. From the [documentation of to_numeric](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_numeric.html), we get:

* If ‘raise’, then invalid parsing will raise an exception
* If ‘coerce’, then invalid parsing will be set as NaN
* If ‘ignore’, then invalid parsing will return the input

``` python

df["deaths"] = pd.to_numeric(df["deaths"], errors='coerce')
df["death_rate"] = pd.to_numeric(df["death_rate"], errors='coerce')
df["age_adjusted_death_rate"] = pd.to_numeric(df["age_adjusted_death_rate"], errors='coerce')
df.dtypes

```

### Last but not least, we can also mark some variables as categorical

``` python

df["sex"] = pd.Categorical(df["sex"])
df["race_ethnicity"] = pd.Categorical(df["race_ethnicity"])
df["leading_cause"] = pd.Categorical(df["leading_cause"])
df.dtypes

```

---

# Exercise 3: What was the leading cause of death in 2014?

``` python

# your code here

```

# Solution

``` python

df2014 = df[df['year'] == 2014]
df2014

```

``` python

df2014['leading_cause'].describe()

```

---

# Exercise 4: How many different causes of death were recorded in 2011?

``` python

# your code here

```

# Solution

``` python

df2014['leading_cause'].nunique()

```

---